# Évaluation qualitative
> Comparons les différents algorithmes en terme de qualité de prédiction et de temps de calcul

# Comparaison entre algorithmes

Charger les bibliothèques et les données

In [96]:
%autosave 0
%reload_ext autoreload
%autoreload 2

from data import *
import popularity
import knn
import knn2
import svd
#import als

from eval import *

Autosave disabled


In [51]:
import sys
sys.path
import data

In [52]:
M = load_data(tiny=True)
M.shape, np.sum(~np.isnan(M))

((50, 40), 389)

## 3.2. KNN : approche memory-based k plus proches voisins

### User-based collaborative filtering (knn)

Liste d'algorithmes à comparer

In [111]:
recommenders_ub_knn = [
    {"fn": popularity.complete, "label": "popularity"},
    {"fn": lambda M_train: knn.complete(M_train, k=1), "label": f"knn1"},
    {"fn": lambda M_train: knn.complete(M_train, k=3), "label": f"knn3"},
    {"fn": lambda M_train: knn.complete(M_train, k=5), "label": f"knn5"},
    {"fn": lambda M_train: knn.complete(M_train, k=7), "label": f"knn7"},
    {"fn": lambda M_train: knn.complete(M_train, k=9), "label": f"knn9"},
    {"fn": lambda M_train: knn.complete(M_train, k=10), "label": f"knn10"},
    {"fn": lambda M_train: knn.complete(M_train, k=15), "label": f"knn15"},
    {"fn": lambda M_train: knn.complete(M_train, k=20), "label": f"knn20"},
    {"fn": lambda M_train: knn.complete(M_train, k=25), "label": f"knn25"},
    {"fn": lambda M_train: knn.complete(M_train, k=30), "label": f"knn30"},
    {"fn": lambda M_train: knn.complete(M_train, k=40), "label": f"knn40"},
    {"fn": lambda M_train: knn.complete(M_train, k=50), "label": f"knn50"}
]

Comparaison

In [54]:
user_based_knn_leaderboard = quantitative_comparison(RMSE, M, recommenders_ub_knn, prop=0.8, nrep=3)
user_based_knn_leaderboard

/home/ensai/Documents/Cours_ENSAI_3A/Rec_systems/TP/repo/mine/popularity.py:40: RuntimeWarning: Mean of empty slice
  scores = np.nanmean(M_train, axis=0)    # compute the average rating for each movies


,recommender,validation score,training score,computation time
0,popularity,1.312026,0.000000,0.000455
1,knn1,1.337462,1.631381,0.675541
2,knn3,1.213324,1.142317,0.739217
3,knn5,1.211400,0.987500,0.814083
4,knn7,1.191549,0.919205,0.787700
5,knn9,1.184549,0.872570,1.252703
6,knn10,1.173776,0.852622,0.749827
7,knn15,1.157693,0.800289,0.805220
8,knn20,1.154527,0.786265,0.748891
9,knn25,1.148655,0.783178,0.630211


La meilleure valeur de k est 25. Elle permet d'obtenir le meilleur score sur l'échantillon de validation (1.148655).

### Item-based collaborative filtering (knn)

Liste d'algorithmes à comparer

In [112]:
recommenders_ib_knn = [
    {"fn": lambda M_train: knn2.complete(M_train, k=1), "label": f"knn2_1"},
    {"fn": lambda M_train: knn2.complete(M_train, k=3), "label": f"knn2_3"},
    {"fn": lambda M_train: knn2.complete(M_train, k=5), "label": f"knn2_5"},
    {"fn": lambda M_train: knn2.complete(M_train, k=7), "label": f"knn2_7"},
    {"fn": lambda M_train: knn2.complete(M_train, k=9), "label": f"knn2_9"},
    {"fn": lambda M_train: knn2.complete(M_train, k=10), "label": f"knn2_10"},
    {"fn": lambda M_train: knn2.complete(M_train, k=15), "label": f"knn2_15"},
    {"fn": lambda M_train: knn2.complete(M_train, k=20), "label": f"knn2_20"},
    {"fn": lambda M_train: knn2.complete(M_train, k=25), "label": f"knn2_25"},
    {"fn": lambda M_train: knn2.complete(M_train, k=30), "label": f"knn2_30"}
]

In [56]:
item_based_knn_leaderboard = quantitative_comparison(RMSE, M[:, 1:], recommenders_ib_knn, prop=0.8, nrep=3)
item_based_knn_leaderboard

,recommender,validation score,training score,computation time
0,knn2_1,1.203240,1.569281,0.581577
1,knn2_3,1.059928,1.192100,0.624341
2,knn2_5,1.041348,1.041334,0.704241
3,knn2_7,1.038990,0.959289,0.665937
4,knn2_9,1.044782,0.897705,0.685661
5,knn2_10,1.046224,0.870014,0.730516
6,knn2_15,1.045143,0.799300,0.678415
7,knn2_20,1.048382,0.769467,0.640338
8,knn2_25,1.040307,0.765439,0.672479
9,knn2_30,1.042131,0.767008,0.665543


La première colonne (film) des données est pleine de `nan`. Par conséquent, même après avoir appliqué la fonction `knn2.complete()`, elle reste inchangée et empêche le calcul des scores de validation. Je supprime donc la colonne concernée avant d'appliquer les modèles basés sur l'approche item-based. 

En outre, la meilleure valeur de k ici est 7, avec un score de validation de 1,03899.

## 3.3. SVD : Factorisation de matrice par décomposition en valeurs singulières

### Grande quantité de données

Charger les bibliothèques et les données

In [74]:
M = load_data(tiny=False)
M.shape, np.sum(~np.isnan(M))

((500, 400), 31553)

On fait varier $k$. Observe-t-on un phénomène de sur-apprentissage ?

In [75]:
recommenders_svd = [
#    {"fn": lambda M_train: knn.complete(M_train, k=10), "label": "knn10"},
    {"fn": lambda M_train: svd.complete(M_train, k=3), "label": "svd3"},
    {"fn": lambda M_train: svd.complete(M_train, k=5), "label": "svd5"},
    {"fn": lambda M_train: svd.complete(M_train, k=8), "label": "svd8"},
    {"fn": lambda M_train: svd.complete(M_train, k=10), "label": "svd10"},
    {"fn": lambda M_train: svd.complete(M_train, k=20), "label": "svd20"},
    {"fn": lambda M_train: svd.complete(M_train, k=50), "label": "svd50"},
    {"fn": lambda M_train: svd.complete(M_train, k=100), "label": "svd100"},
    {"fn": lambda M_train: svd.complete(M_train, k=200), "label": "svd200"},
    {"fn": lambda M_train: svd.complete(M_train, k=400), "label": "svd400"},
    ]
res_fill_with_zeros = quantitative_comparison(RMSE, M, recommenders_svd, prop=0.8, nrep=4)
res_fill_with_zeros

,recommender,validation score,training score,computation time
0,svd3,2.651221,2.574239e+00,0.208200
1,svd5,2.575149,2.453971e+00,0.139622
2,svd8,2.553775,2.337218e+00,0.127798
3,svd10,2.570568,2.283717e+00,0.168516
4,svd20,2.716986,2.076845e+00,0.129136
5,svd50,3.148233,1.605829e+00,0.191183
6,svd100,3.532771,1.038684e+00,0.195464
7,svd200,3.781206,3.800365e-01,0.144218
8,svd400,3.828862,7.880122e-15,0.132540


Oui, on observe ici un phénomène de sur-apprentissage car le score sur les données d'entrainement dimunue jusqu'à atteindre presque 0 (pour k=400) alors que le score sur les données de validation augmente.

La meilleure valeur de $k$ est 8 avec un score de validation de 2.5537

### Quelques recommendations faites par cette méthode

In [80]:
svd.recommend(M, 6, k=8)

58

In [81]:
svd.recommend(M, 155, k=8)

127

In [82]:
svd.recommend(M, 296, k=8)

172

In [83]:
svd.recommend(M, 361, k=8)

272

### Petite quantité de données

In [84]:
M = load_data(tiny=True)
M.shape, np.sum(~np.isnan(M))

((50, 40), 389)

In [113]:
recommenders_svd = [
#    {"fn": lambda M_train: knn.complete(M_train, k=10), "label": "knn10"},
    {"fn": lambda M_train: svd.complete(M_train, k=3), "label": "svd3"},
    {"fn": lambda M_train: svd.complete(M_train, k=5), "label": "svd5"},
    {"fn": lambda M_train: svd.complete(M_train, k=8), "label": "svd8"},
    {"fn": lambda M_train: svd.complete(M_train, k=10), "label": "svd10"},
    {"fn": lambda M_train: svd.complete(M_train, k=20), "label": "svd20"},
    {"fn": lambda M_train: svd.complete(M_train, k=50), "label": "svd50"},
    {"fn": lambda M_train: svd.complete(M_train, k=100), "label": "svd100"},
    {"fn": lambda M_train: svd.complete(M_train, k=200), "label": "svd200"},
    {"fn": lambda M_train: svd.complete(M_train, k=400), "label": "svd400"},
    ]

In [85]:
res_fill_with_zeros = quantitative_comparison(RMSE, M, recommenders_svd, prop=0.8, nrep=4)
res_fill_with_zeros

,recommender,validation score,training score,computation time
0,svd3,3.153008,2.205491e+00,0.000669
1,svd5,3.412829,1.855758e+00,0.000414
2,svd8,3.625572,1.432775e+00,0.000494
3,svd10,3.729769,1.191482e+00,0.000484
4,svd20,3.892747,4.203689e-01,0.000451
5,svd50,3.910568,1.382035e-14,0.000521
6,svd100,3.910568,1.382035e-14,0.000424
7,svd200,3.910568,1.382035e-14,0.000371
8,svd400,3.910568,1.382035e-14,0.000405


Quand tiny=True, on a beaucoup moins de données et l'exécution est très rapide. Par contre, les performances sont moins intéressantes que pour des grandes quantitées de données. 

Par ailleurs, la meilleure valeur de k est 3 avec un score de 3,1530.  

# 4. Métriques d'évaluations

Pour la precision et le recall. Les formules utilisées sont les suivantes

* Precision : 
$$
\text{Precision}_N = \frac{1}{|U|} \sum_{u \in U} \frac{|L_u^N \cap Rel_u|}{|L_u^N|}
$$

* Recall : 
$$
\text{Recall}_N = \frac{1}{|U|} \sum_{u \in U} \frac{|L_u \cap Rel_u|}{|Rel_u|}
$$

Avec :
* $U$ l'ensemble des utilisateurs
* $u$ un utilisateur
* $N$ est le cut-off c'est-à-dire le nombre de recommandations. Ici, nous l'avons mis à 5 par défaut. 

* $Rel_u$ est l'ensemble des **relevant items**. Ici, on a choisi les items qui sont dans l'ensemble de test et qui ont une note supérieure à une certain seuil `th` avec `th=2.5` par défaut. 

* $L_u^N$ est la liste des recommandations. les éléments de cette liste sont choisi dans le **candidate set** $C_u$ (et on a $L_u^N \subseteq C_u$)

Dans notre cas, pour l'ensemble $C_u$, on a chosis l'approche **TestRatings** c'est-à-dire l'ensemble des éléments noté par l'utilisateur dans les données de tests. 

La nouvelle fonction de comparaison prends en compte les nouvelles métriques ajoutées

In [117]:
quantitative_comparison_v2(RMSE, M, recommenders_ub_knn, prop=0.7, nrep=4)

/home/ensai/Documents/Cours_ENSAI_3A/Rec_systems/TP/repo/mine/popularity.py:40: RuntimeWarning: Mean of empty slice
  scores = np.nanmean(M_train, axis=0)    # compute the average rating for each movies


,recommender,RMSE validation,RMSE training,MAE validation,Precision,Recall,computation time
0,popularity,1.174006,0.000000,0.904045,0.512791,0.976283,0.000302
1,knn1,1.257531,1.581307,0.998630,0.510465,0.974409,0.644761
2,knn3,1.224187,1.089195,0.975748,0.508140,0.971583,0.666764
3,knn5,1.211560,0.938886,0.970049,0.510465,0.974199,0.630402
4,knn7,1.169973,0.855711,0.932426,0.510465,0.974199,0.586395
5,knn9,1.182485,0.815767,0.939294,0.511628,0.975653,0.642605
6,knn10,1.176730,0.799303,0.936118,0.511628,0.975653,0.686429
7,knn15,1.147441,0.767063,0.907732,0.511628,0.975653,0.733530
8,knn20,1.148008,0.761580,0.908864,0.511628,0.975653,0.694752
9,knn25,1.147672,0.761588,0.908587,0.511628,0.975653,0.668227


In [118]:
quantitative_comparison_v2(RMSE, M[:, 1:], recommenders_ib_knn, prop=0.7, nrep=4)

/home/ensai/Documents/Cours_ENSAI_3A/Rec_systems/TP/repo/mine/knn2.py:60: RuntimeWarning: Mean of empty slice
  scores[id_user] = np.nanmean(M_train[:, id_item])


,recommender,RMSE validation,RMSE training,MAE validation,Precision,Recall,computation time
0,knn2_1,NaN,1.553471,NaN,0.502960,0.973707,0.514543
1,knn2_3,NaN,1.158077,NaN,0.502960,0.973707,0.538204
2,knn2_5,NaN,0.985296,NaN,0.501823,0.972571,0.576488
3,knn2_7,NaN,0.903037,NaN,0.501797,0.972738,0.608826
4,knn2_9,NaN,0.834902,NaN,0.504123,0.974676,0.668047
5,knn2_10,NaN,0.811545,NaN,0.502986,0.973540,0.648026
6,knn2_15,NaN,0.764975,NaN,0.502960,0.973707,0.592099
7,knn2_20,NaN,0.757378,NaN,0.502960,0.973707,0.614794
8,knn2_25,NaN,0.756952,NaN,0.502960,0.973707,0.566885
9,knn2_30,NaN,0.754273,NaN,0.502960,0.973707,0.636248


In [119]:
quantitative_comparison_v2(RMSE, M, recommenders_svd, prop=0.7, nrep=4)

,recommender,RMSE validation,RMSE training,MAE validation,Precision,Recall,computation time
0,svd3,3.303616,2.292312e+00,3.079190,0.508721,0.974704,0.000725
1,svd5,3.557840,1.918125e+00,3.316925,0.501771,0.970191,0.000772
2,svd8,3.726987,1.466084e+00,3.500544,0.502933,0.971612,0.000817
3,svd10,3.811646,1.196566e+00,3.600176,0.500661,0.969340,0.000868
4,svd20,3.954145,3.923689e-01,3.787780,0.499498,0.968243,0.000742
5,svd50,3.956557,9.220157e-15,3.801471,0.501823,0.970530,0.000659
6,svd100,3.956557,9.220157e-15,3.801471,0.501823,0.970530,0.000862
7,svd200,3.956557,9.220157e-15,3.801471,0.501823,0.970530,0.000818
8,svd400,3.956557,9.220157e-15,3.801471,0.501823,0.970530,0.000883
